# Quickstart

affe quickstart guide.

# Preliminaries

## Imports

In [1]:
import affe
import numpy as np
import pandas as pd

In [2]:
from affe.execs import (
    CompositeExecutor,
    NativeExecutor,
    JoblibExecutor,
    GNUParallelExecutor,
)

In [3]:
from affe.tests import get_dummy_flow

# Flow Definition

Typically, you have some kind of workflow that you want (or need) to deal with, and need to configure and so on.

## Flows saying _"hi"_

To illustrate, let us create 10 different workflows. Each of those says "hi" in a signature way.

In [4]:
# Making a flow is very easy.
flows = [
    get_dummy_flow(message="hi" * (i + 1), content=dict(i=i * 10)) for i in range(3)
]

In [5]:
flow = flows[0]

In [6]:
flow.config

{'io': {'fs': {'root': '/Users/zissou/repos/affe',
   'cli': 'root',
   'data': 'root',
   'out': 'root',
   'scripts': 'root',
   'out.flow.config': 'out.flow',
   'out.flow.logs': 'out.flow',
   'out.flow.results': 'out.flow',
   'out.flow.models': 'out.flow',
   'out.flow.timings': 'out.flow',
   'out.flow.tmp': 'out.flow',
   'out.flow.flows': 'out.flow',
   'out.flow': 'out'}},
 'message': 'hi',
 'content': {'i': 0}}

# Flow Execution

In [ ]:
flow.run()

In [ ]:
flow.run_with_log()

In [ ]:
flows[1].run_with_log()

# Flow Scheduling

= Multiple execution

In [ ]:
e = NativeExecutor
c_jl = JoblibExecutor(flows, e, n_jobs=3)

In [ ]:
c_jl.run()